#### Loading neccessary libraries

In [3]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import timedelta
import holidays
import joblib
import dash
from dash import Dash, dcc, html, Input, Output, callback
from datetime import datetime, date
import dash_bootstrap_components as dbc
from dash import dash_table
from dash.dash_table.Format import Group  
import dash_table
from dash.dependencies import Input, Output, State
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_squared_error, recall_score
from lightgbm import LGBMRegressor

In [4]:
# importing all the saved models
# 7 models for predicting demand for 7 repair priority types based on weather and time of year data
# 23 models for predicting demand for 23 repair trades based on weather and time of year data
# 7 models for predicting demand for 7 repair priority types based on property data
# 23 models for predicting demand for 23 repair trades based on property data

priority_weather_model_filenames = ['PRIORITY Weather Based Models/regressor_priority_routine_model_AUG.joblib', 
                                   'PRIORITY Weather Based Models/regressor_priority_emergency_model_AUG.joblib',
                                   'PRIORITY Weather Based Models/regressor_priority_cyclical_model_AUG.joblib',
                                   'PRIORITY Weather Based Models/regressor_priority_planned_model_AUG.joblib',
                                   'PRIORITY Weather Based Models/regressor_priority_void_model_AUG.joblib',
                                   'PRIORITY Weather Based Models/regressor_priority_inspection_model_AUG.joblib',
                                   'PRIORITY Weather Based Models/regressor_priority_other_model_AUG.joblib',]

trade_weather_model_filenames = ['TRADE weather based models/regressor_plumbing_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_carpentry_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_electrical_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_not_specified_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_decs_issue_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_wet_trades_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_gas_fitter_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_fitter_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_gas_servicing_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_exterior_works_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_general_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_ghs_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_special_quotes_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_metalworker_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_esw_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_pre_termination_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_call_outs_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_decent_homes_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_inspection_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_interior_non_utility_model_sor_wt.joblib',
                                'TRADE weather based models/regressor_specialist_model_sor_wt.joblib',  
                                'TRADE weather based models/regressor_day_rates_model_sor_wt.joblib',  
                                'TRADE weather based models/regressor_asbestos_model_sor_wt.joblib'
                                ]


priority_property_model_filenames = ['PRIORITY Property Based Models/regressor_routine_priority_prop_model.joblib', 
                                     'PRIORITY Property Based Models/regressor_emergency_priority_prop_model.joblib',
                                     'PRIORITY Property Based Models/regressor_cyclical_priority_prop_model.joblib',
                                     'PRIORITY Property Based Models/regressor_planned_priority_prop_model.joblib',
                                     'PRIORITY Property Based Models/regressor_void_priority_prop_model.joblib',
                                     'PRIORITY Property Based Models/regressor_inspection_priority_prop_model.joblib',
                                     'PRIORITY Property Based Models/regressor_other_priority_prop_model.joblib',    
                                   ]

trade_property_model_filenames = ['TRADE property based models/regressor_plumbing_sor_prop_model.joblib',
                                'TRADE property based models/regressor_carpentry_sor_prop_model.joblib',
                                'TRADE property based models/regressor_electrical_sor_prop_model.joblib',
                                'TRADE property based models/regressor_not_specified_sor_prop_model.joblib',
                                'TRADE property based models/regressor_decs_issue_sor_prop_model.joblib',
                                'TRADE property based models/regressor_wet_trades_sor_prop_model.joblib',
                                'TRADE property based models/regressor_gas_fitter_sor_prop_model.joblib',
                                'TRADE property based models/regressor_fitter_sor_prop_model.joblib',
                                'TRADE property based models/regressor_gas_servicing_sor_prop_model.joblib',
                                'TRADE property based models/regressor_exterior_works_sor_prop_model.joblib',
                                'TRADE property based models/regressor_general_sor_prop_model.joblib',
                                'TRADE property based models/regressor_ghs_sor_prop_model.joblib',
                                'TRADE property based models/regressor_special_quotes_sor_prop_model.joblib',
                                'TRADE property based models/regressor_metalworker_sor_prop_model.joblib',
                                'TRADE property based models/regressor_esw_sor_prop_model.joblib',
                                'TRADE property based models/regressor_pre_termination_sor_prop_model.joblib',
                                'TRADE property based models/regressor_call_outs_sor_prop_model.joblib',
                                'TRADE property based models/regressor_decent_homes_sor_prop_model.joblib',
                                'TRADE property based models/regressor_inspection_sor_prop_model.joblib',
                                'TRADE property based models/regressor_interior_non_utility_sor_prop_model.joblib',
                                'TRADE property based models/regressor_specialist_sor_prop_model.joblib',
                                'TRADE property based models/regressor_day_rates_sor_prop_model.joblib',
                                'TRADE property based models/regressor_asbestos_sor_prop_model.joblib',    
                                ]




trade_names = ['Plumbing','Carpentry', 'Electrical', 'Not Specified', 'Decs Issue', 'Wet Trades', 'Gas Fitter', 
               'Fitter', 'Gas Servicing', 'Exterior Works', 'General', 'GHS', 'Special/Quotes', 'Metalworker', 'ESW',  
                'Pre-termination', 'Call Outs', 'Decent Homes', 'Inspection', 'Interior Non Utility',  'Specialist', 
                'Day Rates', 'Asbestos']

models_dict = {}
    
for name in priority_weather_model_filenames:
    model = joblib.load(name)
    models_dict[name] = model

for name in trade_weather_model_filenames:
    model = joblib.load(name)
    models_dict[name] = model

for name in priority_property_model_filenames:
    model = joblib.load(name)
    models_dict[name] = model

for name in trade_property_model_filenames:
    model = joblib.load(name)
    models_dict[name] = model

    
# source for weather descriptions and weather icons                                
github_url = 'https://gist.githubusercontent.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c/raw/7f2d37310ac5d5c309fd9d2f4dd98cc837c28237/descriptions.json'
responsegit = requests.get(github_url)
data_dict = responsegit.json()

# source for GCH logo to insert into Dash App
logo_link = 'https://www.gch.co.uk/wp-content/uploads/2019/01/GCH-Logo-PNG-e1623162177131.png'

## initiating app 
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MORPH])


markdown_text = '''
###### A Web App designed to predict repair demand based on *Weather and Time of Year*, and separately based on *Property Characteristics*.
Please select the start and end date from the date selector, and choose what type of criteria the demand should be split by: _**Priority**_ type or _**Trade**_ type.

For predictions of repair demand based on property characteristics, please make the appropriate selection and enter relevant values, 
before clicking the **SUBMIT** button.

Please allow a minute for the predictions to loapd in tabular format.

'''
# declaring nominal columns
nominal_cols = ['property_class', 'property_size','location_code','quarter_value']

# defining function for onehot encoding used for encoding training data in all models
def one_hot_encoder(data, nominal_cols, drop_nan = True, nan_cols = []):
    nominal_cols = [col for col in nominal_cols if col in data.columns]
    data_ohe = pd.get_dummies(data, columns = nominal_cols)
    if not drop_nan == False:
        for nan_col in nan_cols:
            if nan_col in data_ohe.columns:
                data_ohe = data_ohe.drop(nan_col, axis = 1)
    return data_ohe

# initialise MinMaxScaler object 
scaler = MinMaxScaler()



# building Dash App layout 
app.layout = dbc.Container([
    
  
    dbc.Row([
        dbc.Col(
            
            html.Div([html.H1('GCH Repair Demand Prediction Tool',
                        className='text-primary-emphasis mb-4', style={'font-family': 'Serif'}),
                #width=8,
            ], style={'display': 'flex', 'justify-content': 'flex-end', 'padding-right': '110px'}),
    ),
    ]),
    
    
    dbc.Row([
        dbc.Col([
                
               html.Div([html.Img(src=logo_link, style={'height': '200px', 'width': '300px'})
                        ]),
            
            ]),
        
        
        dbc.Col([
            
            
            dcc.Markdown(markdown_text, style={'font-size': '13px', 'font-family': 'Sans-serif', 'color':'black'})],width=8),]),
    
    
    dbc.Row([
        dbc.Col([
            
            dbc.Form([
            dbc.Label('Display Predictions Based On:', style={'fontSize': 12, 'color': 'black'}),
            dbc.RadioItems(
                id='display-type-radio',
                options=[
                    {'label': 'Weather', 'value': 'Weather'},
                    {'label': 'Property Characteristics', 'value': 'Property Characteristics'}
                ],
                value='Weather',
                inline=True,
                labelStyle={'display': 'inline-block', 'paddingRight': '10px', 'color': 'black'}
            )
        ])], width = 6, align = 'center',)
    ]),
    


    
    dbc.Row([
    dbc.Col([
    html.Div([
        # Date Picker/Calendar for selecting date range
        dcc.DatePickerRange(
            id='my-date-picker-range',
            #date=None,
            start_date = None,
            min_date_allowed=date(1995, 8, 5),
            max_date_allowed=date(2050, 9, 19),
            initial_visible_month=datetime.today(),
            end_date = None,
            display_format='DD/MM/YYYY',
            clearable=True
        )
    ], style={'width': '100%'}),  

        html.Div(id='date-picker-output'),
        ], width={'size': 3, 'offset': 0, 'order': 0}, align='start'),
        
        
        
        # Property Age input
        dbc.Col([
        
            dbc.Form([
                dbc.Label('Property Age:', style={'fontSize': 12, 'color':'black'}),
                dbc.Input(id='property-age-input', type='number', step=1, placeholder='Enter property age', value = 0, disabled = True)
                ]
            ),]),
        
        
        # Property Size input
        dbc.Col([
        dbc.Form([
            dbc.Label('Property Size:', style={'fontSize': 12, 'color': 'black'}),
            dcc.Dropdown(id='property-size-dropdown', options=[
                {'label': 'Studio', 'value':'Studio'},
                {'label':'1 bed', 'value':'1 bed'},
                {'label': '2 bed', 'value':'2 bed'},
                {'label': '3 bed', 'value':'3 bed'},
                {'label': 'Other', 'value':'Other'},], value='None', disabled = True)
        ]),
        ]),
        
        # Property Class input
        dbc.Col([
        dbc.Form([
            dbc.Label('Property Class:', style={'fontSize': 12, 'color': 'black'}),
            dcc.Dropdown(id='property-class-dropdown', options=[
                {'label': 'House', 'value':'House'},
                {'label':'Flat', 'value':'Flat'},
                {'label': 'Block', 'value':'Block'},
                {'label': 'Street', 'value':'Street'},
                {'label': 'Garage', 'value':'Garage'},
                {'label': 'Public Bldg', 'value':'Public Bldg'},
                {'label': 'Commercial', 'value':'Commercial'},
                {'label': 'Virtual', 'value':'Virtual'},], value='None', disabled = True)
        ])]),
        
        # Property Location input
        dbc.Col([
        dbc.Form([
            dbc.Label('Property Location Code:', style={'fontSize': 12, 'color': 'black'}),
            dcc.Dropdown(id='property-location-dropdown', options=[
                {'label': 'QUE', 'value': 'QUE'},
                {'label': 'CHU', 'value': 'CHU'},
                {'label': 'KIN', 'value': 'KIN'},
                {'label': 'GLO', 'value': 'GLO'},
                {'label': 'OVA', 'value': 'OVA'},
                {'label': 'HOL', 'value': 'HOL'},
                {'label': 'MAT', 'value': 'MAT'},
                {'label': 'WES', 'value': 'WES'},
                {'label': 'TUF', 'value': 'TUF'},
                {'label': 'CON', 'value': 'CON'},
                {'label': 'WHI', 'value': 'WHI'},
                {'label': 'ROB', 'value': 'ROB'},
                {'label': 'TRE', 'value': 'TRE'},
                {'label': 'POD', 'value': 'POD'},
                {'label': 'ABB', 'value': 'ABB'},
                {'label': 'LRD', 'value': 'LRD'},
                {'label': 'ELM', 'value': 'ELM'},
                {'label': 'GAR', 'value': 'GAR'},
                {'label': 'BAR', 'value': 'BAR'},
                {'label': 'HUC', 'value': 'HUC'},
                {'label': 'LON', 'value': 'LON'},
                {'label': 'LNG', 'value': 'LNG'},
                {'label': 'ABY', 'value': 'ABY'},
                {'label': 'STO', 'value': 'STO'},
                {'label': 'HIG', 'value': 'HIG'},
                {'label': 'WIT', 'value': 'WIT'},
                {'label': 'HOR', 'value': 'HOR'},
                {'label': 'TWY', 'value': 'TWY'},
                {'label': 'CNM', 'value': 'CNM'},
                {'label': 'HWK', 'value': 'HWK'},
                {'label': 'MAI', 'value': 'MAI'},
                {'label': 'WTM', 'value': 'WTM'},
                {'label': 'BKW', 'value': 'BKW'},
                {'label': 'HEM', 'value': 'HEM'},], value='None', disabled = True)
        ]),
        ]),
        
    ]),
             
    
    dbc.Row([
          
    dbc.Col([
        dbc.Form([
            # Radio button for selecting prediction type - predictions for repair priority types or trade types
            dbc.Label('Prediction Type:', style={'fontSize': 12, 'color': 'black'}),
            dbc.RadioItems(
                id='prediction-type-radio',
                options=[
                    {'label': 'Repair Demand by Repair Priority Type', 'value': 'Repair Demand by Repair Priority Type'},
                    {'label': 'Repair Demand by Trade Type', 'value': 'Repair Demand by Trade Type'}
                ],
                value='Repair Demand by Repair Priority Type',
                inline=True,
                labelStyle={'display': 'inline-block', 'paddingRight': '1px', 'color': 'black'}
            )
        ])
    ], width={'size': 4, 'offset': 0, 'order':1}),
        
    ]),

    dbc.Row([
    # submit button    
    dbc.Col([html.Div([
            html.Button('Submit', id='submit-button', n_clicks=0)]),
            ]),]),
    
    # table for displaying output (for predicted weather and predicted repair count for each category)
    dbc.Row([
    dbc.Col([
        html.Div(id='weather-table', children=[]),
    ]),

    dbc.Col([
        html.Div(id='predictions-container', children=[
            html.Div(id='priority-weather-predictions-table', children=[
                dash_table.DataTable(
                    id='predictions-table-priority-weather',
                    columns=[
                        {'name': 'Date', 'id': 'Date'},
                        {'name': 'Routine Repair Counts', 'id': 'Routine Repair Counts'},
                        {'name': 'Emergency Repair Counts', 'id': 'Emergency Repair Counts'},
                        {'name': 'Cyclical Repair Counts', 'id': 'Cyclical Repair Counts'},
                        {'name': 'Planned Repair Counts', 'id': 'Planned Repair Counts'},
                        {'name': 'Void Repair Counts', 'id': 'Void Repair Counts'},
                        {'name': 'Inspection Counts', 'id': 'Inspection Counts'},
                        {'name': 'Other Counts', 'id': 'Other Counts'},
                    ],
                    data=[],
                    style_table={'overflowX': 'scroll'},
                )
            ], style={'margin-top': '20px'}),

            html.Div(id='trade-weather-predictions-table', children=[
                dash_table.DataTable(
                    id='predictions-table-trade-weather',
                    columns=[
                        {'name': 'Date', 'id': 'Date'},
                        {'name': 'Plumbing Repair Counts', 'id': 'Plumbing Repair Counts'},
                        {'name': 'Carpentry Repair Counts', 'id': 'Carpentry Repair Counts'},
                        {'name': 'Electrical Repair Counts', 'id': 'Electrical Repair Counts'},
                        {'name': 'Not Specified Repair Counts', 'id': 'Not Specified Repair Counts'},
                        {'name': 'Decs Issue Repair Counts', 'id': 'Decs Issue Repair Counts'},
                        {'name': 'Wet Trades Repair Counts', 'id': 'Wet Trades Repair Counts'},
                        {'name': 'Gas Fitter Repair Counts', 'id': 'Gas Fitter Repair Counts'},
                        {'name': 'Fitter Repair Counts', 'id': 'Fitter Repair Counts'},
                        {'name': 'Gas Servicing Repair Counts', 'id': 'Gas Servicing Repair Counts'},
                        {'name': 'Exterior Works Repair Counts', 'id': 'Exterior Works Repair Counts'},
                        {'name': 'General Repair Counts', 'id': 'General Repair Counts'},
                        {'name': 'GHS Repair Counts', 'id': 'GHS Repair Counts'},
                        {'name': 'Special/Quotes Repair Counts', 'id': 'Special/Quotes Repair Counts'},
                        {'name': 'Metalworker Repair Counts', 'id': 'Metalworker Repair Counts'},
                        {'name': 'ESW Repair Counts', 'id': 'ESW Repair Counts'},
                        {'name': 'Pre-termination Repair Counts', 'id': 'Pre-termination Repair Counts'},
                        {'name': 'Call Outs Repair Counts', 'id': 'Call Outs Repair Counts'},
                        {'name': 'Decent Homes Repair Counts', 'id': 'Decent Homes Repair Counts'},
                        {'name': 'Inspection Repair Counts', 'id': 'Inspection Repair Counts'},
                        {'name': 'Interior Non Utility Repair Counts', 'id': 'Interior Non Utility Repair Counts'},
                        {'name': 'Specialist Repair Counts', 'id': 'Specialist Repair Counts'},
                        {'name': 'Day Rates Repair Counts', 'id': 'Day Rates Repair Counts'},
                        {'name': 'Asbestos Repair Counts', 'id': 'Asbestos Repair Counts'},
                    ],
                    data=[],
                    style_table={'overflowX': 'scroll'},
                )
            ], style={'margin-top': '20px'}),

            html.Div(id='priority-property-predictions-table', children=[
                dash_table.DataTable(
                    id='predictions-table-priority-property',
                    columns=[
                        {'name': 'Year', 'id': 'Year'},
                        {'name': 'Quarter', 'id': 'Quarter'},
                        {'name': 'Routine Repair Counts', 'id': 'Routine Repair Counts'},
                        {'name': 'Emergency Repair Counts', 'id': 'Emergency Repair Counts'},
                        {'name': 'Cyclical Repair Counts', 'id': 'Cyclical Repair Counts'},
                        {'name': 'Planned Repair Counts', 'id': 'Planned Repair Counts'},
                        {'name': 'Void Repair Counts', 'id': 'Void Repair Counts'},
                        {'name': 'Inspection Counts', 'id': 'Inspection Counts'},
                        {'name': 'Other Counts', 'id': 'Other Counts'},
                    ],
                    data=[],
                    style_table={'overflowX': 'scroll'},
                )
            ], style={'margin-top': '20px'}),

            html.Div(id='trade-property-predictions-table', children=[
                dash_table.DataTable(
                    id='predictions-table-trade-property',
                    columns=[
                        {'name': 'Year', 'id': 'Year'},
                        {'name': 'Quarter', 'id': 'Quarter'},
                        {'name': 'Plumbing Repair Counts', 'id': 'Plumbing Repair Counts'},
                        {'name': 'Carpentry Repair Counts', 'id': 'Carpentry Repair Counts'},
                        {'name': 'Electrical Repair Counts', 'id': 'Electrical Repair Counts'},
                        {'name': 'Not Specified Repair Counts', 'id': 'Not Specified Repair Counts'},
                        {'name': 'Decs Issue Repair Counts', 'id': 'Decs Issue Repair Counts'},
                        {'name': 'Wet Trades Repair Counts', 'id': 'Wet Trades Repair Counts'},
                        {'name': 'Gas Fitter Repair Counts', 'id': 'Gas Fitter Repair Counts'},
                        {'name': 'Fitter Repair Counts', 'id': 'Fitter Repair Counts'},
                        {'name': 'Gas Servicing Repair Counts', 'id': 'Gas Servicing Repair Counts'},
                        {'name': 'Exterior Works Repair Counts', 'id': 'Exterior Works Repair Counts'},
                        {'name': 'General Repair Counts', 'id': 'General Repair Counts'},
                        {'name': 'GHS Repair Counts', 'id': 'GHS Repair Counts'},
                        {'name': 'Special/Quotes Repair Counts', 'id': 'Special/Quotes Repair Counts'},
                        {'name': 'Metalworker Repair Counts', 'id': 'Metalworker Repair Counts'},
                        {'name': 'ESW Repair Counts', 'id': 'ESW Repair Counts'},
                        {'name': 'Pre-termination Repair Counts', 'id': 'Pre-termination Repair Counts'},
                        {'name': 'Call Outs Repair Counts', 'id': 'Call Outs Repair Counts'},
                        {'name': 'Decent Homes Repair Counts', 'id': 'Decent Homes Repair Counts'},
                        {'name': 'Inspection Repair Counts', 'id': 'Inspection Repair Counts'},
                        {'name': 'Interior Non Utility Repair Counts', 'id': 'Interior Non Utility Repair Counts'},
                        {'name': 'Specialist Repair Counts', 'id': 'Specialist Repair Counts'},
                        {'name': 'Day Rates Repair Counts', 'id': 'Day Rates Repair Counts'},
                        {'name': 'Asbestos Repair Counts', 'id': 'Asbestos Repair Counts'},
                    ],
                    data=[],
                    style_table={'overflowX': 'scroll'},
                )
            ], style={'margin-top': '20px'}),
        ],style={'maxWidth': '800px'})
    ]),
]), 

],)
    


# callback section: connecting all the components
# ************************************************************************
    
# defining function for fetching/scrapping weather data from Open Weather API        
def fetch_weather_data(latitude, longitude, start_date, end_date):
    
    api_url = f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&daily=weathercode,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,shortwave_radiation_sum&timezone=Europe%2FLondon&start_date={start_date}&end_date={end_date}'
    weather_request = requests.get(api_url)
    if weather_request.status_code == 200:
        json_data = weather_request.json()
        weather_data = json_data['daily']
        
        # specifying which columns (weather features in weather forecast) to extract from the weather APi
        columns_to_extract = ['time', 'temperature_2m_min', 'temperature_2m_max', 'precipitation_sum', 'weathercode', 'rain_sum', 'snowfall_sum', 'precipitation_hours', 'windspeed_10m_max', 'windgusts_10m_max', 'shortwave_radiation_sum']
        
        # storing extracted data in dictionary
        extracted_data = {column: weather_data[column] for column in columns_to_extract}

        # storing extracted data in dataframe
        new_weather_df = pd.DataFrame(extracted_data)

        return new_weather_df
    else:
        print('Wrong input or API error')
        return None
    


# declaring callbacks for App
# keeping property features disabled for user input when 'Display Type = Weather' is selected
@app.callback(
    [Output('property-size-dropdown', 'disabled'), 
     Output('property-age-input', 'disabled'),
     Output('property-location-dropdown', 'disabled'),
     Output('property-class-dropdown', 'disabled')],  
     Input('display-type-radio', 'value')
)
def toggle_property_inputs(display_type):
    if display_type == 'Weather':
        return True, True, True, True
    else:
        return False, False, False, False


    
# returning start and end date based on values selected from Date Picker    
@app.callback(
    Output('my-date-picker-range', 'start_date'),
    Output('my-date-picker-range', 'end_date'),
    Input('my-date-picker-range', 'start_date'),
    Input('my-date-picker-range', 'end_date')
)
def update_date_range(start_date, end_date):
    today = date.today()
    if start_date and datetime.strptime(start_date, '%Y-%m-%d').date() < today:
        start_date = today.strftime('%Y-%m-%d')
    if end_date and datetime.strptime(end_date, '%Y-%m-%d').date() < today:
        end_date = today.strftime('%Y-%m-%d')
    return start_date, end_date
    
# returning Dash table to display weather predictions for selected dates, along with repair demand predictions for those dates
@app.callback(

    Output('priority-weather-predictions-table', 'children'),
    Output('trade-weather-predictions-table', 'children'),
    Output('weather-table', 'children'),
    Output('priority-property-predictions-table', 'children'),
    Output('trade-property-predictions-table', 'children'),
    Input('submit-button', 'n_clicks'),
    State('display-type-radio', 'value'),
    State('prediction-type-radio', 'value'),
    State('my-date-picker-range', 'start_date'),
    State('my-date-picker-range', 'end_date'),
    State('property-age-input', 'value'),
    State('property-size-dropdown', 'value'),
    State('property-class-dropdown', 'value'),
    State('property-location-dropdown', 'value')
    
 
)


def update_data_table(n_clicks, display_type, prediction_type, start_date, end_date, property_age, property_size, property_class, location_code):
    
    # initiate weather table
    weather_table = html.Div()
    
    # if there is no click yet, return default values or an empty output
    if n_clicks is None:
        
        return [], [],weather_table ,[],[]
    
    
    if start_date is None or end_date is None or property_age is None or property_size is None or property_class is None or location_code is None:
        return [], [], weather_table,[],[]
    
    radio_button_mapping = {
        'Repair Demand by Repair Priority Type': priority_weather_model_filenames,
        'Repair Demand by Trade Type': trade_weather_model_filenames
    }
    # mapping model keys to respective variable names (for connecting with dispay tables)
    weather_priority_key_mapping = {
        'weather_PRIORITY Weather Based Models/regressor_priority_routine_model_AUG.joblib_predictions': 'Routine Repair Counts',
        'weather_PRIORITY Weather Based Models/regressor_priority_emergency_model_AUG.joblib_predictions': 'Emergency Repair Counts',
        'weather_PRIORITY Weather Based Models/regressor_priority_cyclical_model_AUG.joblib_predictions': 'Cyclical Repair Counts',
        'weather_PRIORITY Weather Based Models/regressor_priority_planned_model_AUG.joblib_predictions': 'Planned Repair Counts',
        'weather_PRIORITY Weather Based Models/regressor_priority_void_model_AUG.joblib_predictions': 'Void Repair Counts',
        'weather_PRIORITY Weather Based Models/regressor_priority_inspection_model_AUG.joblib_predictions': 'Inspection Counts',
        'weather_PRIORITY Weather Based Models/regressor_priority_other_model_AUG.joblib_predictions': 'Other Counts'
    }
    
    weather_trade_key_mapping = {
    'weather_TRADE weather based models/regressor_plumbing_model_sor_wt.joblib_predictions': 'Plumbing Repair Counts',
    'weather_TRADE weather based models/regressor_carpentry_model_sor_wt.joblib_predictions': 'Carpentry Repair Counts',
    'weather_TRADE weather based models/regressor_electrical_model_sor_wt.joblib_predictions': 'Electrical Repair Counts',
    'weather_TRADE weather based models/regressor_not_specified_model_sor_wt.joblib_predictions': 'Not Specified Repair Counts',
    'weather_TRADE weather based models/regressor_decs_issue_model_sor_wt.joblib_predictions': 'Decs Issue Repair Counts',
    'weather_TRADE weather based models/regressor_wet_trades_model_sor_wt.joblib_predictions': 'Wet Trades Repair Counts',
    'weather_TRADE weather based models/regressor_gas_fitter_model_sor_wt.joblib_predictions': 'Gas Fitter Repair Counts',
    'weather_TRADE weather based models/regressor_fitter_model_sor_wt.joblib_predictions': 'Fitter Repair Counts',
    'weather_TRADE weather based models/regressor_gas_servicing_model_sor_wt.joblib_predictions': 'Gas Servicing Repair Counts',
    'weather_TRADE weather based models/regressor_exterior_works_model_sor_wt.joblib_predictions': 'Exterior Works Repair Counts',
    'weather_TRADE weather based models/regressor_general_model_sor_wt.joblib_predictions': 'General Repair Counts',
    'weather_TRADE weather based models/regressor_ghs_model_sor_wt.joblib_predictions': 'GHS Repair Counts',
    'weather_TRADE weather based models/regressor_special_quotes_model_sor_wt.joblib_predictions': 'Special/Quotes Repair Counts',
    'weather_TRADE weather based models/regressor_metalworker_model_sor_wt.joblib_predictions': 'Metalworker Repair Counts',
    'weather_TRADE weather based models/regressor_esw_model_sor_wt.joblib_predictions': 'ESW Repair Counts',
    'weather_TRADE weather based models/regressor_pre_termination_model_sor_wt.joblib_predictions': 'Pre-termination Repair Counts',
    'weather_TRADE weather based models/regressor_call_outs_model_sor_wt.joblib_predictions': 'Call Outs Repair Counts',
    'weather_TRADE weather based models/regressor_decent_homes_model_sor_wt.joblib_predictions': 'Decent Homes Repair Counts',
    'weather_TRADE weather based models/regressor_inspection_model_sor_wt.joblib_predictions': 'Inspection Repair Counts',
    'weather_TRADE weather based models/regressor_interior_non_utility_model_sor_wt.joblib_predictions': 'Interior Non Utility Repair Counts',
    'weather_TRADE weather based models/regressor_specialist_model_sor_wt.joblib_predictions': 'Specialist Repair Counts',
    'weather_TRADE weather based models/regressor_day_rates_model_sor_wt.joblib_predictions': 'Day Rates Repair Counts',
    'weather_TRADE weather based models/regressor_asbestos_model_sor_wt.joblib_predictions': 'Asbestos Repair Counts'
}
    
    property_priority_key_mapping = {

        
    'property_PRIORITY Property Based Models/regressor_routine_priority_prop_predictions': 'Routine Repair Counts',
    'property_PRIORITY Property Based Models/regressor_emergency_priority_prop_predictions': 'Emergency Repair Counts',
    'property_PRIORITY Property Based Models/regressor_cyclical_priority_prop_predictions': 'Cyclical Repair Counts',
    'property_PRIORITY Property Based Models/regressor_planned_priority_prop_predictions': 'Planned Repair Counts',
    'property_PRIORITY Property Based Models/regressor_void_priority_prop_predictions': 'Void Repair Counts',
    'property_PRIORITY Property Based Models/regressor_inspection_priority_prop_predictions': 'Inspection Counts',
    'property_PRIORITY Property Based Models/regressor_other_priority_prop_predictions': 'Other Counts'
    }
    
    property_trade_key_mapping = {
    'property_TRADE property based models/regressor_plumbing_sor_prop_predictions': 'Plumbing Repair Counts',
    'property_TRADE property based models/regressor_carpentry_sor_prop_predictions': 'Carpentry Repair Counts',
    'property_TRADE property based models/regressor_electrical_sor_prop_predictions': 'Electrical Repair Counts',
    'property_TRADE property based models/regressor_not_specified_sor_prop_predictions': 'Not Specified Repair Counts',
    'property_TRADE property based models/regressor_decs_issue_sor_prop_predictions': 'Decs Issue Repair Counts',
    'property_TRADE property based models/regressor_wet_trades_sor_prop_predictions': 'Wet Trades Repair Counts',
    'property_TRADE property based models/regressor_gas_fitter_sor_prop_predictions': 'Gas Fitter Repair Counts',
    'property_TRADE property based models/regressor_fitter_sor_prop_predictions': 'Fitter Repair Counts',
    'property_TRADE property based models/regressor_gas_servicing_sor_prop_predictions': 'Gas Servicing Repair Counts',
    'property_TRADE property based models/regressor_exterior_works_sor_prop_predictions': 'Exterior Works Repair Counts',
    'property_TRADE property based models/regressor_general_sor_prop_predictions': 'General Repair Counts',
    'property_TRADE property based models/regressor_ghs_sor_prop_predictions': 'GHS Repair Counts',
    'property_TRADE property based models/regressor_special_quotes_sor_prop_predictions': 'Special/Quotes Repair Counts',
    'property_TRADE property based models/regressor_metalworker_sor_prop_predictions': 'Metalworker Repair Counts',
    'property_TRADE property based models/regressor_esw_sor_prop_predictions': 'ESW Repair Counts',
    'property_TRADE property based models/regressor_pre_termination_sor_prop_predictions': 'Pre-termination Repair Counts',
    'property_TRADE property based models/regressor_call_outs_sor_prop_predictions': 'Call Outs Repair Counts',
    'property_TRADE property based models/regressor_decent_homes_sor_prop_predictions': 'Decent Homes Repair Counts',
    'property_TRADE property based models/regressor_inspection_sor_prop_predictions': 'Inspection Repair Counts',
    'property_TRADE property based models/regressor_interior_non_utility_sor_prop_predictions': 'Interior Non Utility Repair Counts',
    'property_TRADE property based models/regressor_specialist_sor_prop_predictions': 'Specialist Repair Counts',
    'property_TRADE property based models/regressor_day_rates_sor_prop_predictions': 'Day Rates Repair Counts',
    'property_TRADE property based models/regressor_asbestos_sor_prop_predictions': 'Asbestos Repair Counts'
}
    # radio buttons logic
    if prediction_type == 'Repair Demand by Repair Priority Type':
        
        if display_type == 'Weather':
            current_key_mapping = weather_priority_key_mapping
            current_model_filenames = priority_weather_model_filenames
            current_table_id = 'predictions-table-priority-weather'
        elif display_type == 'Property Characteristics':
            current_key_mapping = property_priority_key_mapping
            current_model_filenames = priority_property_model_filenames
            current_table_id = 'predictions-table-priority-property'
        else:
            return [], [],weather_table,[],[]
       

    elif prediction_type == 'Repair Demand by Trade Type':
        
        if display_type == 'Weather':
            current_key_mapping = weather_trade_key_mapping
            current_model_filenames = trade_weather_model_filenames
            current_table_id = 'predictions-table-trade-weather'
        elif display_type == 'Property Characteristics':
            current_key_mapping = property_trade_key_mapping
            current_model_filenames = trade_property_model_filenames
            current_table_id = 'predictions-table-trade-property'
        else:
            return [], [],weather_table,[],[]

       
        print(f'spredictions table trade')
   
    else:
        return [], [],weather_table,[],[]
    
    
    # initiating empty predictions data (to be later dispalyed)
    data = []
    

    if end_date is not None:
        
        
        selected_datetime = datetime.strptime(start_date, '%Y-%m-%d').date()
        
        print(f'START DATE IS :{selected_datetime}')
        today = date.today()
        
        # weather predictions are available only 16 days into the future (free weather API)
        max_end_date = today + timedelta(days=15)       
        end_date_dt = datetime.strptime(end_date, '%Y-%m-%d').date() if end_date else None
        
        if end_date_dt > max_end_date:
            end_date_dt = max_end_date
            end_date = end_date_dt.strftime('%Y-%m-%d') 

        else:
            end_date = end_date

        if selected_datetime >= today:
            days_left = (selected_datetime - today).days
            if days_left <= 16:
                latitude = 51.8657
                longitude = -2.2431
                #print('end date', end_date)
                weather_forecast = fetch_weather_data(latitude, longitude, start_date, end_date)
                #print(weather_forecast)
                if weather_forecast is not None:
                    uk_holidays = holidays.UnitedKingdom()
                    
                    
                    # adding our engineered features based on 'time' (as used for training models)
                    weather_forecast['Year'] = pd.to_datetime(weather_forecast['time']).dt.year
                    weather_forecast['Week'] = pd.to_datetime(weather_forecast['time']).dt.isocalendar().week
                    weather_forecast['Month'] = pd.to_datetime(weather_forecast['time']).dt.month
                    weather_forecast['Day'] = pd.to_datetime(weather_forecast['time']).dt.day
                    weather_forecast['WeekDay'] = pd.to_datetime(weather_forecast['time']).dt.dayofweek
                    weather_forecast['Holiday'] = weather_forecast['time'].isin(uk_holidays)
                    weather_forecast['BeginMonth'] = weather_forecast['Day'].isin([1, 2, 3]).astype(int)
                    weather_forecast['Weekend'] = weather_forecast['WeekDay'].isin([5, 6]).astype(int)
                    
                    # calculating mean temperature based on max and min temperature (mean not available in weather forecast)
                    weather_forecast['temperature_2m_mean'] = (weather_forecast['temperature_2m_min'] + weather_forecast['temperature_2m_max']) / 2
                    weather_forecast['Holiday'] = weather_forecast['Holiday'].apply(int) 
                    weather_forecast['Week'] = weather_forecast['Week'].astype(int)

                    # correctng column order to be according to training data
                    column_order = ['time','Year', 'Week', 'Month', 'Day', 'WeekDay', 'Holiday', 'BeginMonth', 'Weekend',
                                    'temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean',
                                    'shortwave_radiation_sum', 'precipitation_sum', 'rain_sum', 'snowfall_sum',
                                    'precipitation_hours', 'windspeed_10m_max', 'windgusts_10m_max']
                    
                    weather_forecast = weather_forecast[column_order]
                    # specifying feature names to be exactly according to those in training data
                    column_names_units = [
                        'time','Year', 'Week', 'Month', 'Day', 'WeekDay', 'Holiday', 'BeginMonth', 'Weekend',
                        'temperature_2m_max (°C)', 'temperature_2m_min (°C)', 'temperature_2m_mean (°C)',
                        'shortwave_radiation_sum (MJ/m²)', 'precipitation_sum (mm)', 'rain_sum (mm)',
                        'snowfall_sum (cm)', 'precipitation_hours (h)', 'windspeed_10m_max (km/h)',
                        'windgusts_10m_max (km/h)'
                    ]

                    # updating column names
                    weather_forecast.columns = column_names_units
                    display(weather_forecast)
                    
                    # dictioanry for storing predictions for each date
                    predictions_dict = {}

                    # logic for predictions based on 'Weather' and 'Time of Year' data (Objective 1)
                    if display_type == 'Weather':
                        predictions_row = []
                        for num in range(len(weather_forecast['time'])):
                            new_dict = {}
                            new_dict['Date'] = weather_forecast['time'][num]
                            predictions_row.append(new_dict)
                    
                    # logic for predictions based on 'Property' data (Objective 1)
                    elif display_type == 'Property Characteristics':
                        predictions_row = []
                        for num in range(len(weather_forecast['time'])):
                            new_dict = {}
                            new_dict['Year'] = pd.to_datetime(weather_forecast['time'][num]).year
                            quarter = pd.to_datetime(weather_forecast['time'][num]).quarter
                            new_dict['Quarter'] = f'Q{quarter}'
                            predictions_row.append(new_dict)

                    # load models from current model filenames selected based on 'Prediction Type' and 'Display type'
                    for model_filename in current_model_filenames:
                        model = models_dict[model_filename]

                        # logic for predictions based on Weather and Time of Year data (Objective 1)    
                        if display_type == 'Weather':
                            print('display weather')

                            day_info = {}
                            # getting weather descriptions and icon for selected dates to display alongside repair predictions
                            for key, value in data_dict.items():
                                day_description = value['day']['description']
                                image_link = value['day']['image']
                                day_info[key] = {'description': day_description, 'image': image_link}

                            wet = fetch_weather_data(latitude, longitude, start_date, end_date)
                            wet['weathercode description'] = wet['weathercode'].astype(str).map(lambda x: day_info[x]['description'])
                            wet['weathercode icon'] = wet['weathercode'].astype(str).map(lambda x: day_info[x]['image'])

                            weather_data_des = wet[['time','weathercode description','weathercode icon']]


                            weather_table = html.Div([
                                        html.H4('Weather Condition'),
                                        html.Table([
                                            html.Thead([
                                                html.Tr([
                                                    html.Th('Date'),
                                                    html.Th('Weather Description'),
                                                    html.Th('Weather Icon')
                                                ])
                                            ]),
                                            html.Tbody([
                                                html.Tr([
                                                    html.Td(row['time']),
                                                    html.Td(row['weathercode description']),
                                                    html.Td(html.Img(src=row['weathercode icon']))
                                                ]) for _, row in weather_data_des.iterrows()
                                            ])
                                        ],style={'font-size': '12px', 'padding': '1px'})
                                    ], style={'maxHeight': '2000px','maxWidth': '300px'})


                            # logic for classifier models (we did not end up selecting any classifier models - initially we chose classfifers for predicting demand for minority classes)
                            if 'classifier' in model_filename:

                                predictions = model.predict(weather_forecast.drop('time', axis=1)) 
                                predictions_df = pd.DataFrame(predictions, columns=['repair_count'])
                                model_name = model_filename.replace('_model.joblib', '')
                                predictions_dict[f'weather_{model_name}_predictions'] = predictions_df
                                old_preds = predictions
                                
                            # logic for regressor models
                            else:

                                # making predictions using the weather forecast data
                                predictions = model.predict(weather_forecast.drop('time', axis = 1))
                                #print(predictions)
                                # inverse log transform (since we regress on log-transformed target - repair counts)
                                pred_real = np.expm1(predictions)
                                rounded_pred = np.round(pred_real).astype(int)
                                rounded_pred[rounded_pred < 0] = 0

                                # storing predictions in dataframe 
                                predictions_df = pd.DataFrame(rounded_pred, columns=['repair_count'])

                                # specifying model name according to model filename (remove '_model.joblib')
                                model_name = model_filename.replace('_model.joblib', '')

                                # stroing predictions dataframe in predictions_dict with the corresponding model name
                                predictions_dict[f'weather_{model_name}_predictions'] = predictions_df
                                old_preds = rounded_pred
                                #print(old_preds)
                        
                        # logic for predictions based on Property Year data (Objective 2)  
                        elif display_type == 'Property Characteristics':
                            
                            # not displaying weathrt for predictions based on property data (empty)
                            weather_table = html.Div()
                            # getting predictions for the same number of dates as selected from Date Range Picker
                            new_num = len(weather_forecast['time'])

                            next_16_days = [selected_datetime + timedelta(days=i) for i in range(new_num)]
                            next_16_days_str = [date.strftime('%Y-%m-%d') for date in next_16_days]


                            # creating dataframe to store property data for the next 16 days
                            property_data = pd.DataFrame({
                                'Date': next_16_days_str,
                                #'Date':selected_datetime.strftime('%Y-%m-%d'),
                                'property_class': [property_class] * new_num,
                                'property_size': [property_size] * new_num,
                                'location_code': [location_code] * new_num,
                                'property_age': [property_age] * new_num,
                            })

                            #print(f'property DATA:{property_data}')
                            # getting property data ready according to training data used for training property based models
                            property_data['Year'] = pd.to_datetime(property_data['Date']).dt.year
                            property_data['Quarter'] = pd.to_datetime(property_data['Date']).dt.to_period('Q')
                            property_data['Quarter'] = property_data['Quarter'].astype(str)
                            property_data['quarter_value'] = property_data['Quarter'].str.extract(r'Q(\d+)')
                            property_data['quarter_value'] = property_data['Quarter'].str.extract(r'Q(\d+)').apply(lambda x: 'Q' + x)  
                            property_data = property_data.drop('Quarter', axis = 1)

                            column_order_pr = ['Year', 'quarter_value', 'property_class', 'property_size','location_code', 'property_age']

                            property_data = property_data[column_order_pr]
                            print(f'property DATA:{property_data}')
                            
                            property_data_encoded = one_hot_encoder(property_data, nominal_cols)

                            all_encoded_columns = ['Year', 'property_age', 'property_class_Block',
                                                   'property_class_Commercial', 'property_class_Flat',
                                                   'property_class_Garage', 'property_class_House',
                                                   'property_class_Public Bldg', 'property_class_Street',
                                                   'property_class_Virtual', 'property_size_1 Bed', 'property_size_2 Bed',
                                                   'property_size_3 Bed', 'property_size_4 Bed', 'property_size_5 Bed',
                                                   'property_size_6 Bed', 'property_size_Other', 'property_size_Other ',
                                                   'property_size_Studio', 'location_code_ABB', 'location_code_BAR',
                                                   'location_code_BKW', 'location_code_CNM', 'location_code_CON',
                                                   'location_code_ELM', 'location_code_GAR', 'location_code_GLO',
                                                   'location_code_HEM', 'location_code_HIG', 'location_code_HOL',
                                                    'location_code_HOR', 'location_code_HUC', 'location_code_HWK',
                                                    'location_code_KIN', 'location_code_LNG', 'location_code_LON',
                                                    'location_code_LRD', 'location_code_MAI', 'location_code_MAT',
                                                    'location_code_OVA', 'location_code_POD', 'location_code_QUE',
                                                    'location_code_ROB', 'location_code_STO', 'location_code_STR',
                                                    'location_code_TRE', 'location_code_TUF', 'location_code_TWY',
                                                    'location_code_WES', 'location_code_WHI', 'location_code_WIT',
                                                    'location_code_WTM', 'quarter_value_Q1', 'quarter_value_Q2',
                                                    'quarter_value_Q3', 'quarter_value_Q4']

                            for column in all_encoded_columns:
                                if column not in property_data_encoded.columns:
                                    property_data_encoded[column] = 0


                            property_data_encoded = property_data_encoded[all_encoded_columns]

                            #print(f'property ENCODED DATA:{property_data_encoded}')
                            display(property_data_encoded)
                            property_data_normalized = scaler.fit_transform(property_data_encoded)
                            #print(f'property DATA:{property_data}')
                            #print(f'property_data_NORMALIZED DATA:{property_data_normalized}')

                            # logic for classifier models used for predicting repair demand based on Property Characteristics
                            if 'classifier' in model_filename:
                                #print('display prop class')
                                predictions = model.predict(property_data_normalized)
                                #preds = [item[0] for item in predictions]
                                predictions_df = pd.DataFrame(predictions, columns=['repair_count'])
                                model_name = model_filename.replace('_model.joblib', '')
                                predictions_dict[f'property_{model_name}_predictions'] = predictions_df
                                #print(predictions)
                                if isinstance(predictions[0], np.ndarray):
                                    
                                    new_preds = []
                                    for pred in predictions:
                                        new_preds.append(pred[0])
                                    old_preds = new_preds
                                else:
                                    old_preds = predictions
                            else:
                                #print('display prop class else')
                                predictions = model.predict(property_data_normalized)
                                pred_real = np.expm1(predictions)
                                rounded_pred = np.round(pred_real).astype(int)
                                rounded_pred[rounded_pred < 0] = 0

                                predictions_df = pd.DataFrame(rounded_pred, columns=['repair_count'])
                                #print(f'property predictions:{predictions_df}')
                                model_name = model_filename.replace('_model.joblib', '')
                                
                                # storing predictions in predictions_dict with corresponding model name
                                predictions_dict[f'property_{model_name}_predictions'] = predictions_df
                                #print(predictions)
                                old_preds = rounded_pred

                        for model_filename_var, predictions_df in predictions_dict.items():
                            if model_name in model_filename_var:
                                new_key = current_key_mapping[model_filename_var]

                                #print(f'Model Name: {model_name}')
                                #print(f'Model preds: {predictions_df}')

                                predictions_df.set_index('repair_count')
                                #print(predictions_df)
                                predictions = predictions_df['repair_count']
                                for num in range(len(old_preds)):
                                    predictions_row[num][new_key] = old_preds[num]
                                #print(predictions_row)

                    # updating repair demand predictions data for every date
                    data = predictions_row

                    #print(f'data:{data}')

                    df_dictionary = pd.DataFrame(predictions_row)
                    display(df_dictionary)
                    
                    # saving to csv for sharing results with GCH
                    #new_predictions_to_send = df_dictionary.to_csv(r'C:\Users\adeel\OneDrive\Desktop\pred_new.csv')

                    # logic for displaying final predictions through Dash Tables 
                    if prediction_type == 'Repair Demand by Repair Priority Type':
                        #print('pred type rep')                                                             
                        if display_type == 'Weather':
                            #print('pred type rep weath')                                                             
                            #print('prediction type is PRIORITY RADIO BUTTON')
                            
                            priority_columns = [
                                {'name': 'Date', 'id': 'Date'},
                                {'name': 'Routine Repair Counts', 'id': 'Routine Repair Counts'},
                                {'name': 'Emergency Repair Counts', 'id': 'Emergency Repair Counts'},
                                {'name': 'Cyclical Repair Counts', 'id': 'Cyclical Repair Counts'},
                                {'name': 'Planned Repair Counts', 'id': 'Planned Repair Counts'},
                                {'name': 'Void Repair Counts', 'id': 'Void Repair Counts'},
                                {'name': 'Inspection Counts', 'id': 'Inspection Counts'},
                                {'name': 'Other Counts', 'id': 'Other Counts'},
                            ]
                            priority_weather_table = dash_table.DataTable(
                                id=current_table_id,
                                columns=priority_columns,
                                data=data,
                                style_table={'overflowX': 'scroll'},
                            )

                            #print(priority_table)

                            trade_weather_table = None
                            priority_property_table = None
                            trade_property_table = None
                            
                            return priority_weather_table, trade_weather_table, weather_table, priority_property_table, trade_property_table
                        
                        elif display_type == 'Property Characteristics':
                            #print('pred disp type prop')                                                                                     
                            #print('prediction type is PRIORITY RADIO BUTTON')
                            
                            priority_property_columns = [
                                {'name': 'Year', 'id': 'Year'},
                                {'name': 'Quarter', 'id': 'Quarter'},
                                {'name': 'Routine Repair Counts', 'id': 'Routine Repair Counts'},
                                {'name': 'Emergency Repair Counts', 'id': 'Emergency Repair Counts'},
                                {'name': 'Cyclical Repair Counts', 'id': 'Cyclical Repair Counts'},
                                {'name': 'Planned Repair Counts', 'id': 'Planned Repair Counts'},
                                {'name': 'Void Repair Counts', 'id': 'Void Repair Counts'},
                                {'name': 'Inspection Counts', 'id': 'Inspection Counts'},
                                {'name': 'Other Counts', 'id': 'Other Counts'},
                            ]
                            priority_property_table = dash_table.DataTable(
                                id=current_table_id,
                                columns=priority_property_columns,
                                data=data,
                                style_table={'overflowX': 'scroll'},
                                page_size=1
                            )

                            trade_weather_table = None
                            priority_weather_table = None
                            trade_property_table = None
                            
                            return priority_weather_table, trade_weather_table, weather_table, priority_property_table, trade_property_table
                        

                    elif prediction_type == 'Repair Demand by Trade Type':
                        
                        #print('prediction type is TRADE RADIO BUTTON')
                        
                        if display_type == 'Weather':
                            #print('trade weather')
                            priority_weather_table = None
                            #trade_weather_table = None
                            priority_property_table = None
                            trade_property_table = None

                            trade_columns = [
                                {'name': 'Date', 'id': 'Date'},
                                {'name': 'Plumbing Repair Counts', 'id': 'Plumbing Repair Counts'},
                                {'name': 'Carpentry Repair Counts', 'id': 'Carpentry Repair Counts'},
                                {'name': 'Electrical Repair Counts', 'id': 'Electrical Repair Counts'},
                                {'name': 'Not Specified Repair Counts', 'id': 'Not Specified Repair Counts'},
                                {'name': 'Decs Issue Repair Counts', 'id': 'Decs Issue Repair Counts'},
                                {'name': 'Wet Trades Repair Counts', 'id': 'Wet Trades Repair Counts'},
                                {'name': 'Gas Fitter Repair Counts', 'id': 'Gas Fitter Repair Counts'},
                                {'name': 'Fitter Repair Counts', 'id': 'Fitter Repair Counts'},
                                {'name': 'Gas Servicing Repair Counts', 'id': 'Gas Servicing Repair Counts'},
                                {'name': 'Exterior Works Repair Counts', 'id': 'Exterior Works Repair Counts'},
                                {'name': 'General Repair Counts', 'id': 'General Repair Counts'},
                                {'name': 'GHS Repair Counts', 'id': 'GHS Repair Counts'},
                                {'name': 'Special/Quotes Repair Counts', 'id': 'Special/Quotes Repair Counts'},
                                {'name': 'Metalworker Repair Counts', 'id': 'Metalworker Repair Counts'},
                                {'name': 'ESpW Repair Counts', 'id': 'ESW Repair Counts'},
                                {'name': 'Pre-termination Repair Counts', 'id': 'Pre-termination Repair Counts'},
                                {'name': 'Call Outs Repair Counts', 'id': 'Call Outs Repair Counts'},
                                {'name': 'Decent Homes Repair Counts', 'id': 'Decent Homes Repair Counts'},
                                {'name': 'Inspection Repair Counts', 'id': 'Inspection Repair Counts'},
                                {'name': 'Interior Non Utility Repair Counts', 'id': 'Interior Non Utility Repair Counts'},
                                {'name': 'Specialist Repair Counts', 'id': 'Specialist Repair Counts'},
                                {'name': 'Day Rates Repair Counts', 'id': 'Day Rates Repair Counts'},
                                {'name': 'Asbestos Repair Counts', 'id': 'Asbestos Repair Counts'},
                            ]  
                            trade_weather_table = dash_table.DataTable(
                                id=current_table_id,
                                columns=trade_columns,
                                data=data,
                                style_table={'overflowX': 'scroll'},
                            )
                            #print(trade_table)

                            return priority_weather_table, trade_weather_table, weather_table, priority_property_table, trade_property_table
                        
                        elif display_type == 'Property Characteristics':
                            #print('prop char')
                            priority_weather_table = None
                            trade_weather_table = None
                            priority_property_table = None
                            
                            trade_property_columns = [
                                {'name': 'Year', 'id': 'Year'},
                                {'name': 'Quarter', 'id': 'Quarter'},
                                {'name': 'Plumbing Repair Counts', 'id': 'Plumbing Repair Counts'},
                                {'name': 'Carpentry Repair Counts', 'id': 'Carpentry Repair Counts'},
                                {'name': 'Electrical Repair Counts', 'id': 'Electrical Repair Counts'},
                                {'name': 'Not Specified Repair Counts', 'id': 'Not Specified Repair Counts'},
                                {'name': 'Decs Issue Repair Counts', 'id': 'Decs Issue Repair Counts'},
                                {'name': 'Wet Trades Repair Counts', 'id': 'Wet Trades Repair Counts'},
                                {'name': 'Gas Fitter Repair Counts', 'id': 'Gas Fitter Repair Counts'},
                                {'name': 'Fitter Repair Counts', 'id': 'Fitter Repair Counts'},
                                {'name': 'Gas Servicing Repair Counts', 'id': 'Gas Servicing Repair Counts'},
                                {'name': 'Exterior Works Repair Counts', 'id': 'Exterior Works Repair Counts'},
                                {'name': 'General Repair Counts', 'id': 'General Repair Counts'},
                                {'name': 'GHS Repair Counts', 'id': 'GHS Repair Counts'},
                                {'name': 'Special/Quotes Repair Counts', 'id': 'Special/Quotes Repair Counts'},
                                {'name': 'Metalworker Repair Counts', 'id': 'Metalworker Repair Counts'},
                                {'name': 'ESW Repair Counts', 'id': 'ESW Repair Counts'},
                                {'name': 'Pre-termination Repair Counts', 'id': 'Pre-termination Repair Counts'},
                                {'name': 'Call Outs Repair Counts', 'id': 'Call Outs Repair Counts'},
                                {'name': 'Decent Homes Repair Counts', 'id': 'Decent Homes Repair Counts'},
                                {'name': 'Inspection Repair Counts', 'id': 'Inspection Repair Counts'},
                                {'name': 'Interior Non Utility Repair Counts', 'id': 'Interior Non Utility Repair Counts'},
                                {'name': 'Specialist Repair Counts', 'id': 'Specialist Repair Counts'},
                                {'name': 'Day Rates Repair Counts', 'id': 'Day Rates Repair Counts'},
                                {'name': 'Asbestos Repair Counts', 'id': 'Asbestos Repair Counts'},
                            ]  
                            trade_property_table = dash_table.DataTable(
                                id=current_table_id,
                                columns=trade_property_columns,
                                data=data,
                                style_table={'overflowX': 'scroll'},
                                page_size=1
                            )
                            #print(trade_table)

                            return priority_weather_table, trade_weather_table, weather_table, priority_property_table, trade_property_table
                        
                            

                    else: 
                        print('Failed to fetch weather forecast data.',)
                        return (
                            
                           None, None, weather_table, None, None
                            
                        )
                else:
                    print('Weather prediction data is only available up to 16 days into the future.')
                    return (
                        
                        [],[], weather_table,[],[]
                        
                    )
            else:
                print('Please select a date in the future.')
                return (
                    
                    [],[], weather_table, [],[]
                    
                )

        return data, [], [], weather_table,[],[]
                

                
if __name__=='__main__':
    app.run_server(debug=True, use_reloader=False)
    
print(f'Server is running at: http://127.0.0.1:8050/')

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Server is running at: http://127.0.0.1:8050/


### References: 

https://github.com/Coding-with-Adam/Dash-by-Plotly

https://gist.github.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c